# **Test Data Prep - Google Analytics Customer Revenue Prediction**

[Import libraries and other setup](#section1)

[Load data and flatten json blobs](#section2)

[Delete non useful columns based on train](#section3)

[Delete non useful columns](#section4)

[Fill up missing values](#section5)
- [Columns having single value other than nulls](#section6)
- [Columns having multiple values other than nulls](#section7)

[New Features](#section8)
- [Add 7 new columns similar to train](#section6)

[Save the cleaned test data to csv](#section9)

In [2]:
# should be at top
import project_path

# import the necessary libraries
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from collections import defaultdict

# setup 
#sns.set_style(style="whitegrid")
sns.set_style('ticks')

# display charts inline
%matplotlib inline

# to display all columns
pd.set_option('display.max_columns', None)

# import lib functions
from lib import flatten_json_fields
from lib.boolean_to_one_zero import boolean_to_one_zero
from lib.get_time_fields import get_time_fields

/Neha/Study/DS/SpringBoard/DataScience/Capstone_Second


In [3]:
test_file_path = os.path.abspath(path='../data/test.csv')

In [4]:
%%time

# load the test data for the submission to kaggle
test_df = flatten_json_fields.load_df(test_file_path)

Loaded test.csv. Shape: (804684, 53)
CPU times: user 2min 9s, sys: 6.04 s, total: 2min 15s
Wall time: 2min 15s


In [5]:
test_df.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20171016,6167871330617112363,6167871330617112363_1508151024,Not Socially Engaged,1508151024,2,1508151024,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,Asia,Singapore,not available in demo dataset,not available in demo dataset,(not set),myrepublic.com.sg,not available in demo dataset,(not set),Southeast Asia,NaN,4,NaN,4,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google
1,Organic Search,20171016,0643697640977915618,0643697640977915618_1508175522,Not Socially Engaged,1508175522,1,1508175522,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,Zaragoza,not available in demo dataset,Europe,Spain,not available in demo dataset,not available in demo dataset,(not set),rima-tde.net,not available in demo dataset,Aragon,Southern Europe,NaN,5,1,5,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
2,Organic Search,20171016,6059383810968229466,6059383810968229466_1508143220,Not Socially Engaged,1508143220,1,1508143220,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Europe,France,not available in demo dataset,not available in demo dataset,not available in demo dataset,sfr.net,not available in demo dataset,not available in demo dataset,Western Europe,NaN,7,1,7,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
3,Organic Search,20171016,2376720078563423631,2376720078563423631_1508193530,Not Socially Engaged,1508193530,1,1508193530,Safari,not available in demo dataset,not available in demo dataset,mobile,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in 

In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804684 entries, 0 to 804683
Data columns (total 53 columns):
channelGrouping                                      804684 non-null object
date                                                 804684 non-null int64
fullVisitorId                                        804684 non-null object
sessionId                                            804684 non-null object
socialEngagementType                                 804684 non-null object
visitId                                              804684 non-null int64
visitNumber                                          804684 non-null int64
visitStartTime                                       804684 non-null int64
device.browser                                       804684 non-null object
device.browserSize                                   804684 non-null object
device.browserVersion                                804684 non-null object
device.deviceCategory                                8046

#### Notice that we are missing 1 column trafficSource.campaignCode other than the transaction revenue column. So we can't use that as feature in our train data set. We will remove that from our train data set as well.

#### Columns with 1 unique value and zero null value in the train set

In [7]:
cols_to_delete_df = pd.read_csv("../data/cols_to_delete.csv",index_col=0)

In [8]:
cols_to_delete = cols_to_delete_df.values.flatten().tolist()

In [9]:
len(cols_to_delete)

19

In [10]:
test_df.drop(cols_to_delete, axis=1, inplace=True)

In [11]:
test_df.shape

(804684, 34)

#### Columns that had single value other than nulls in our train set.

In [12]:
# metadata - number of unique values and null values for each column
info = pd.DataFrame(index=test_df.columns.values,data={'data type': test_df.dtypes, \
                          'unique values': test_df.nunique(), 'null values': test_df.isnull().sum()})

In [13]:
info.sort_values(['unique values','null values'])

,data type,unique values,null values
totals.newVisits,object,1,200314
totals.bounces,object,1,383736
trafficSource.isTrueDirect,object,1,544171
trafficSource.adwordsClickInfo.isVideoAd,object,1,750870
device.isMobile,bool,2,0
device.deviceCategory,object,3,0
trafficSource.adwordsClickInfo.adNetworkType,object,3,750870
trafficSource.adwordsClickInfo.slot,object,3,750870
geoNetwork.continent,object,6,0
trafficSource.medium,object,7,0


In [14]:
# convert nans to either 0 or 1
boolean_to_one_zero(test_df, 'totals.bounces',1,True)
boolean_to_one_zero(test_df, 'totals.newVisits',1,True)
boolean_to_one_zero(test_df, 'trafficSource.adwordsClickInfo.isVideoAd',False,False)
#boolean_to_one_zero('trafficSource.campaignCode','11251kjhkvahf',False)
boolean_to_one_zero(test_df, 'trafficSource.isTrueDirect',True,True)

In [15]:
# multi valued columns and having nulls
# categorical columns here would have to be one hot encoded later on or binned in some way
for col in info[(info['unique values']>1) & (info['null values'] > 0)].index:
    print(col, test_df[col].nunique(), test_df[col].isnull().sum())

totals.pageviews 160 139
trafficSource.adContent 51 750893
trafficSource.adwordsClickInfo.adNetworkType 3 750870
trafficSource.adwordsClickInfo.gclId 41317 750822
trafficSource.adwordsClickInfo.page 10 750870
trafficSource.adwordsClickInfo.slot 3 750870
trafficSource.keyword 2415 391032
trafficSource.referralPath 2197 569361


In [16]:
# fillna and make it int
test_df['totals.pageviews'].fillna(0, inplace=True)
test_df['totals.pageviews'] = test_df['totals.pageviews'].astype(int)

# hits to int
test_df['totals.hits'] = test_df['totals.hits'].astype(int)

# fillna with string 'Others'
test_df['trafficSource.adContent'].fillna('Others', inplace=True)

# fillna with string 'Unknown'
#train_df['trafficSource.adwordsClickInfo.adNetworkType'].fillna('Unknown', inplace=True)

# fillna with string 'Unknown'
#train_df['trafficSource.adwordsClickInfo.gclId'].fillna('Unknown', inplace=True)

# trafficSource.adwordsClickInfo.page
# fillna and make it int
test_df['trafficSource.adwordsClickInfo.page'].fillna(0, inplace=True)
test_df['trafficSource.adwordsClickInfo.page'] = test_df['trafficSource.adwordsClickInfo.page'].astype(int)

# trafficSource.adwordsClickInfo.slot, trafficSource.keyword,trafficSource.referralPath
more_cols = ['trafficSource.adwordsClickInfo.adNetworkType','trafficSource.adwordsClickInfo.gclId',\
             'trafficSource.adwordsClickInfo.slot',\
            'trafficSource.keyword','trafficSource.referralPath']
for col in more_cols:
    print(test_df[col].unique()[0:10])
    test_df[col].fillna('Unknown', inplace=True)    

[nan 'Google Search' 'Content' 'Search partners']
[nan
 'Cj0KCQjwsZHPBRClARIsAC-VMPBHdNF2oMOgh6Xp6YhjXWSk11sDu3eCo4De_u6l4xafqgrPVPSGYdkaArkiEALw_wcB'
 'EAIaIQobChMIgsXEgfz11gIVA16GCh2ZgQqUEAAYASAAEgIb4_D_BwE'
 'Cj0KCQjwsZHPBRClARIsAC-VMPDlLD6kS4tmqFGZjMUqyerwe0WDaULaAKNkj-gwuvPnLSPRyf9TN7IaArICEALw_wcB'
 'Cj0KCQjwsZHPBRClARIsAC-VMPD0ZYMPmWCxeRqNz_1Vzn67tHxph1HEn1NgAraEHc9drP29fB4dEs0aAjLSEALw_wcB'
 'EAIaIQobChMIko-r8cz11gIVgrrACh1x5QzVEAAYASAAEgITVvD_BwE'
 'Cj0KCQjwsZHPBRClARIsAC-VMPCDjcuyfjXG2gMJPnyN_uicNBAL5ebDBDxiXRqyFDmdpE2hRMrnd6AaAuRsEALw_wcB'
 'Cj0KCQjwsZHPBRClARIsAC-VMPBv9X_YKr1Llp3ebjZ8fGYQ3J8k1D-ObeGiyZoQqJkiDw2mlBEOGkQaAnohEALw_wcB'
 'Cj0KCQjwsZHPBRClARIsAC-VMPAogovYcJ91-Mkr4qWUXIIRcwhT0u86i3SjlvaJXdTE5k-UUntJqMkaAgZFEALw_wcB'
 'EAIaIQobChMIp4G16-v11gIVihDTCh1ePQc5EAAYASAAEgKm0vD_BwE']
[nan 'Top' 'RHS' 'Google Display Network']
['(not provided)' '6qEhsCssdK0z36ri' nan 'google online merchandise'
 'google company store' 'water bottle' '(Remarketing/Content targeting)'
 '(aut

In [17]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804684 entries, 0 to 804683
Data columns (total 34 columns):
channelGrouping                                 804684 non-null object
date                                            804684 non-null int64
fullVisitorId                                   804684 non-null object
sessionId                                       804684 non-null object
visitId                                         804684 non-null int64
visitNumber                                     804684 non-null int64
visitStartTime                                  804684 non-null int64
device.browser                                  804684 non-null object
device.deviceCategory                           804684 non-null object
device.isMobile                                 804684 non-null bool
device.operatingSystem                          804684 non-null object
geoNetwork.city                                 804684 non-null object
geoNetwork.continent                       

In [18]:
test_df[['date','visitStartTime']]

,date,visitStartTime
0,20171016,1508151024
1,20171016,1508175522
2,20171016,1508143220
3,20171016,1508193530
4,20171016,1508217442
5,20171016,1508186358
6,20171016,1508203650
7,20171016,1508184397
8,20171016,1508189652
9,20171016,1508175502


In [19]:
time_fields = test_df['visitStartTime'].apply(get_time_fields)
time_fields

,year,month,day,weekday,hour
0,2017,10,16,0,10
1,2017,10,16,0,17
2,2017,10,16,0,8
3,2017,10,16,0,22
4,2017,10,17,1,5
5,2017,10,16,0,20
6,2017,10,17,1,1
7,2017,10,16,0,20
8,2017,10,16,0,21
9,2017,10,16,0,17


In [20]:
time_fields.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804684 entries, 0 to 804683
Data columns (total 5 columns):
year       804684 non-null int64
month      804684 non-null int64
day        804684 non-null int64
weekday    804684 non-null int64
hour       804684 non-null int64
dtypes: int64(5)
memory usage: 30.7 MB


In [21]:
test_df = test_df.join(time_fields)
test_df

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,year,month,day,weekday,hour
0,Organic Search,20171016,6167871330617112363,6167871330617112363_1508151024,1508151024,2,1508151024,Chrome,desktop,False,Macintosh,(not set),Asia,Singapore,(not set),myrepublic.com.sg,(not set),Southeast Asia,0,4,0,4,Others,Unknown,Unknown,1,0,Unknown,(not set),1,(not provided),organic,Unknown,google,2017,10,16,0,10
1,Organic Search,20171016,0643697640977915618,0643697640977915618_1508175522,1508175522,1,1508175522,Chrome,desktop,False,Windows,Zaragoza,Europe,Spain,(not set),rima-tde.net,Aragon,Southern Europe,0,5,1,5,Others,Unknown,Unknown,1,0,Unknown,(not set),0,(not provided),organic,Unknown,google,2017,10,16,0,17
2,Organic Search,20171016,6059383810968229466,6059383810968229466_1508143220,1508143220,1,1508143220,Chrome,desktop,False,Macintosh,not available in demo dataset,Europe,France,not available in demo dataset,sfr.net,not available in demo dataset,Western Europe,0,7,1,7,Others,Unknown,Unknown,1,0,Unknown,(not set),0,(not provided),organic,Unknown,google,2017,10,16,0,8
3,Organic Search,20171016,2376720078563423631,2376720078563423631_1508193530,1508193530,1,1508193530,Safari,mobile,True,iOS,Mountain View,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,0,8,1,4,Others,Unknown,Unknown,1,0,Unknown,(not set),0,(not provided),organic,Unknown,google,2017,10,16,0,22
4,Organic Search,20171016,2314544520795440038,2314544520795440038_1508217442,1508217442,1,1508217442,Safari,desktop,False,Macintosh,San Jose,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,0,9,1,4,Others,Unknown,Unknown,1,0,Unknown,(not set),0,(not provided),organic,Unknown,google,2017,10,17,1,5
5,Organic Search,20171016,4133039884103392367,4133039884103392367_1508186358,1508186358,1,1508186358,Chrome,desktop,False,Linux,not available in demo dataset,Americas,United States,not available in demo dataset,(not set),not available in demo dataset,Northern America,0,11,1,5,Others,Unknown,Unknown,1,0,Unknown,(not set),0,(not provided),organic,Unknown,google,2017,10,16,0,20
6,Organic Search,20171016,4320478850207397557,4320478850207397557_1508203650,1508203650,1,1508203650,Chrome,desktop,False,Macintosh,not available in demo dataset,Americas,United States,not available in demo dataset,grandenetworks.net,not available in demo dataset,Northern America,0,37,1,15,Others,Unknown,Unknown,1,0,Unknown,(not set),0,(not provided),organic,Unknown,google,2017,10,17,1,1
7,Organic Search,20171016,5876438247590157131,5876438247590157131_1508184397,1508184397,1,1508184397,Chrome,desktop,False,Windows,not available in demo dataset,Americas,United States,not available in demo dataset,lightpath.net,not available in demo dataset,Northern America,0,52,1,22,Others,Unknown,Unknown,1,0,Unknown,(not set),0,(not provided),organic,Unknown,google,2017,10,16,0,20
8,Organic Search,20171016,0514591268737702944,0514591268737702944_1508189652,1508189652,6,1508189652,Chrome,desktop,False,Macintosh,Lisbon,Europe,Portugal,(not set),pt,Lisbon,Southern Europe,0,5,0,5,Others,Unknown,Unknown,1,0,Unknown,(not set),1,(not provided),organic,Unknown,google,2017,10,16,0,21
9,Organic Search,20171016,6430567031531677212,6430567031531677212_1508175502,1508175502,1,1508175502,Chrome,desktop,False

#### Second set of columns to be dropped

In [22]:
# columns to drop set 2
cols_to_drop = []

# id fields serve no purpose
#cols_to_drop.append('fullVisitorId')
cols_to_drop.append('sessionId')
cols_to_drop.append('visitId')

# since we have data only for a year it is not useful
cols_to_drop.append('year')

# date is redundant because visitStartTime includes date
cols_to_drop.append('date')

# visitStartTime is broken into day, month etc. and exact time upto seconds doesn't make business sense
cols_to_drop.append('visitStartTime')

# continent is redundant becuase sub-continent is bit more granular but includes continent info
cols_to_drop.append('geoNetwork.continent')

# isMobile is redundant, device category already have that category
cols_to_drop.append('device.isMobile')

test_df.drop(cols_to_drop, axis=1, inplace=True)

In [23]:
test_df.shape

(804684, 32)

In [24]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804684 entries, 0 to 804683
Data columns (total 32 columns):
channelGrouping                                 804684 non-null object
fullVisitorId                                   804684 non-null object
visitNumber                                     804684 non-null int64
device.browser                                  804684 non-null object
device.deviceCategory                           804684 non-null object
device.operatingSystem                          804684 non-null object
geoNetwork.city                                 804684 non-null object
geoNetwork.country                              804684 non-null object
geoNetwork.metro                                804684 non-null object
geoNetwork.networkDomain                        804684 non-null object
geoNetwork.region                               804684 non-null object
geoNetwork.subContinent                         804684 non-null object
totals.bounces                        

In [25]:
# day_of_the_year param is doubtful ....needs to be revisited

In [26]:
# save till now
checkpoint_1_df = test_df.copy()

In [27]:
# comment it later
# test_df = checkpoint_1_df.copy()

In [28]:
with open('../data/unique_cat_values.json', encoding='utf-8') as data_file:
    cat_unique_values = json.loads(data_file.read())

In [29]:
type(cat_unique_values)

dict

In [30]:
# pretty printing of json-formatted string
print(json.dumps(cat_unique_values, sort_keys=True, indent=4))

{
    "channelGrouping": [
        "Organic Search",
        "Referral",
        "Paid Search",
        "Affiliates",
        "Direct",
        "dummy_channelGrouping",
        "Social"
    ],
    "device.browser": [
        "Chrome",
        "Firefox",
        "dummy_device_browser",
        "Internet Explorer",
        "Safari",
        "Edge"
    ],
    "device.deviceCategory": [
        "desktop",
        "mobile",
        "tablet"
    ],
    "device.operatingSystem": [
        "Windows",
        "Macintosh",
        "Linux",
        "Android",
        "iOS",
        "Chrome OS",
        "dummy_device_operatingSystem"
    ],
    "geoNetwork.city": [
        "dummy_geoNetwork_city",
        "not available in demo dataset",
        "(not set)",
        "London",
        "Mountain View",
        "New York",
        "San Francisco",
        "Sunnyvale",
        "San Jose"
    ],
    "geoNetwork.country": [
        "Turkey",
        "Australia",
        "Spain",
        "Indonesia",
   

#### Reduce the category unique values to same as train data

In [31]:
for key in cat_unique_values.keys():
    values = cat_unique_values.get(key)
    dummy_val = ('dummy_'+str(key)).replace('.','_')
    indices = test_df[~test_df[key].isin(values)].index
    print(indices.size)
    test_df.loc[indices,key] = dummy_val

45038
68859
0
9379
239705
145710
63460
362522
222907
24683
53791
42224
53862
42323
50313
61040
2815
145929
48699


In [32]:
test_df['geoNetwork.city'].unique()

array(['(not set)', 'dummy_geoNetwork_city',
       'not available in demo dataset', 'Mountain View', 'San Jose',
       'San Francisco', 'Sunnyvale', 'New York', 'London'], dtype=object)

In [33]:
test_df.shape

(804684, 32)

In [34]:
test_df.head()

,channelGrouping,fullVisitorId,visitNumber,device.browser,device.deviceCategory,device.operatingSystem,geoNetwork.city,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,month,day,weekday,hour
0,Organic Search,6167871330617112363,2,Chrome,desktop,Macintosh,(not set),dummy_geoNetwork_country,(not set),dummy_geoNetwork_networkDomain,(not set),Southeast Asia,0,4,0,4,Others,Unknown,Unknown,1,0,Unknown,(not set),1,(not provided),organic,Unknown,google,10,16,0,10
1,Organic Search,0643697640977915618,1,Chrome,desktop,Windows,dummy_geoNetwork_city,Spain,(not set),dummy_geoNetwork_networkDomain,dummy_geoNetwork_region,Southern Europe,0,5,1,5,Others,Unknown,Unknown,1,0,Unknown,(not set),0,(not provided),organic,Unknown,google,10,16,0,17
2,Organic Search,6059383810968229466,1,Chrome,desktop,Macintosh,not available in demo dataset,France,not available in demo dataset,dummy_geoNetwork_networkDomain,not available in demo dataset,Western Europe,0,7,1,7,Others,Unknown,Unknown,1,0,Unknown,(not set),0,(not provided),organic,Unknown,google,10,16,0,8
3,Organic Search,2376720078563423631,1,Safari,mobile,iOS,Mountain View,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,0,8,1,4,Others,Unknown,Unknown,1,0,Unknown,(not set),0,(not provided),organic,Unknown,google,10,16,0,22
4,Organic Search,2314544520795440038,1,Safari,desktop,Macintosh,San Jose,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,0,9,1,4,Others,Unknown,Unknown,1,0,Unknown,(not set),0,(not provided),organic,Unknown,google,10,17,1,5


In [35]:
# quarter
test_df['quarter'] = test_df['month'].apply(lambda x: 'quarter'+str(np.mod(x,4)))

In [36]:
test_df['week_day_end'] = test_df['weekday'].apply(lambda x: 'week_day' if x<5 else 'weekend')

In [37]:
test_df.drop(['weekday'], axis=1, inplace=True)

#### generate dummies for category variables

In [38]:
X_cat = test_df.select_dtypes(include=[object])
X_cat.columns

Index(['channelGrouping', 'fullVisitorId', 'device.browser',
       'device.deviceCategory', 'device.operatingSystem', 'geoNetwork.city',
       'geoNetwork.country', 'geoNetwork.metro', 'geoNetwork.networkDomain',
       'geoNetwork.region', 'geoNetwork.subContinent',
       'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.keyword', 'trafficSource.medium',
       'trafficSource.referralPath', 'trafficSource.source', 'quarter',
       'week_day_end'],
      dtype='object')

In [39]:
category_columns = list(X_cat.columns)
category_columns.extend(['month','day','hour'])
category_columns.remove('fullVisitorId')
category_columns

['channelGrouping',
 'device.browser',
 'device.deviceCategory',
 'device.operatingSystem',
 'geoNetwork.city',
 'geoNetwork.country',
 'geoNetwork.metro',
 'geoNetwork.networkDomain',
 'geoNetwork.region',
 'geoNetwork.subContinent',
 'trafficSource.adContent',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'trafficSource.adwordsClickInfo.gclId',
 'trafficSource.adwordsClickInfo.slot',
 'trafficSource.campaign',
 'trafficSource.keyword',
 'trafficSource.medium',
 'trafficSource.referralPath',
 'trafficSource.source',
 'quarter',
 'week_day_end',
 'month',
 'day',
 'hour']

In [58]:
X_future = pd.get_dummies(test_df, columns=category_columns)

In [64]:
X_future.shape

(804684, 208)

In [66]:
X_future.head()

,fullVisitorId,visitNumber,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.isTrueDirect,channelGrouping_Affiliates,channelGrouping_Direct,channelGrouping_Organic Search,channelGrouping_Paid Search,channelGrouping_Referral,channelGrouping_Social,channelGrouping_dummy_channelGrouping,device.browser_Chrome,device.browser_Edge,device.browser_Firefox,device.browser_Internet Explorer,device.browser_Safari,device.browser_dummy_device_browser,device.deviceCategory_desktop,device.deviceCategory_mobile,device.deviceCategory_tablet,device.operatingSystem_Android,device.operatingSystem_Chrome OS,device.operatingSystem_Linux,device.operatingSystem_Macintosh,device.operatingSystem_Windows,device.operatingSystem_dummy_device_operatingSystem,device.operatingSystem_iOS,geoNetwork.city_(not set),geoNetwork.city_London,geoNetwork.city_Mountain View,geoNetwork.city_New York,geoNetwork.city_San Francisco,geoNetwork.city_San Jose,geoNetwork.city_Sunnyvale,geoNetwork.city_dummy_geoNetwork_city,geoNetwork.city_not available in demo dataset,geoNetwork.country_Australia,geoNetwork.country_Brazil,geoNetwork.country_Canada,geoNetwork.country_France,geoNetwork.country_Germany,geoNetwork.country_India,geoNetwork.country_Indonesia,geoNetwork.country_Italy,geoNetwork.country_Japan,geoNetwork.country_Mexico,geoNetwork.country_Netherlands,geoNetwork.country_Philippines,geoNetwork.country_Poland,geoNetwork.country_Russia,geoNetwork.country_Spain,geoNetwork.country_Taiwan,geoNetwork.country_Thailand,geoNetwork.country_Turkey,geoNetwork.country_United Kingdom,geoNetwork.country_United States,geoNetwork.country_Vietnam,geoNetwork.country_dummy_geoNetwork_country,geoNetwork.metro_(not set),geoNetwork.metro_London,geoNetwork.metro_Los Angeles CA,geoNetwork.metro_New York NY,geoNetwork.metro_San Francisco-Oakland-San Jose CA,geoNetwork.metro_dummy_geoNetwork_metro,geoNetwork.metro_not available in demo dataset,geoNetwork.networkDomain_(not set),geoNetwork.networkDomain_comcast.net,geoNetwork.networkDomain_comcastbusiness.net,geoNetwork.networkDomain_dummy_geoNetwork_networkDomain,geoNetwork.networkDomain_rr.com,geoNetwork.networkDomain_ttnet.com.tr,geoNetwork.networkDomain_unknown.unknown,geoNetwork.networkDomain_verizon.net,geoNetwork.region_(not set),geoNetwork.region_California,geoNetwork.region_England,geoNetwork.region_New York,geoNetwork.region_dummy_geoNetwork_region,geoNetwork.region_not available in demo dataset,geoNetwork.subContinent_Australasia,geoNetwork.subContinent_Central America,geoNetwork.subContinent_Eastern Asia,geoNetwork.subContinent_Eastern Europe,geoNetwork.subContinent_Northern America,geoNetwork.subContinent_Northern Europe,geoNetwork.subContinent_South America,geoNetwork.subContinent_Southeast Asia,geoNetwork.subContinent_Southern Asia,geoNetwork.subContinent_Southern Europe,geoNetwork.subContinent_Western Asia,geoNetwork.subContinent_Western Europe,geoNetwork.subContinent_dummy_geoNetwork_subContinent,trafficSource.adContent_Others,trafficSource.adContent_dummy_trafficSource_adContent,trafficSource.adwordsClickInfo.adNetworkType_Google Search,trafficSource.adwordsClickInfo.adNetworkType_Unknown,trafficSource.adwordsClickInfo.adNetworkType_dummy_trafficSource_adwordsClickInfo_adNetworkType,trafficSource.adwordsClickInfo.gclId_Unknown,trafficSource.adwordsClickInfo.gclId_dummy_trafficSource_adwordsClickInfo_gclId,trafficSource.adwordsClickInfo.slot_Top,trafficSource.adwordsClickInfo.slot_Unknown,trafficSource.adwordsClickInfo.slot_dummy_trafficSource_adwordsClickInfo_slot,trafficSource.campaign_(not set),trafficSource.campaign_AW - Dynamic Search Ads Whole Site,trafficSource.campaign_Data Share Promo,trafficSource.campaign_dummy_trafficSource_campaign,trafficSource.keyword_(not provided),trafficSource.keyword_6qEhsCssdK0z36ri,trafficSource.keyword_Unknown,trafficSource.keyword_dummy_trafficSource_keyword,trafficSource.medium_(none),traffic

In [72]:
X_future.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804684 entries, 0 to 804683
Columns: 208 entries, fullVisitorId to hour_23
dtypes: int64(8), object(1), uint8(199)
memory usage: 208.0+ MB


In [71]:
X_future.to_csv('../data/X_future.csv')

In [70]:
X_future['fullVisitorId'].nunique()

617242